# Hypothesis Testing

This notebook is going to cover **hypothesis testing**, which measures whether a finding is likely to have occurred by chance rather than because of a hypothesized variable. Think of testing a new drug that is supposed to reduce the duration of a cold. If the average time of recovery with the drug is less than the average time without the drug, it has to be enough of a difference so that it's unlikely to have been due to random chance. It could very well be the drug had no effect. 

Let's get some intuition first by exploring the most critical building block for this reasoning: the p-value.

## P-Value Intuition

When somebody says something is statistically significant, what does that mean? It is a phrase thrown around a lot, but few people stop and articulate how statistical significance works on a mathematical level. To understand statistical significance, we can go back to the invention of the p-value in 1925. 

A mathematician named Ronald Fisher was at a party. One of his colleagues Muriel Bristol claimed she could detect when tea was poured before milk simply by tasting it. Intrigued by the claim, Ronald set up an experiment on the spot.

He prepared eight cups of tea. Four had milk poured first; the other four had tea poured first. He then asked her to identify the pour order of each. Remarkably, she identified them all correctly, and if she was simply guessing the probability of this happening by chance is 1 in 70, or 0.01428571. This is what we call the **p-value**, the probablity something happened by random luck rather than because of a hypothesized explanation. 

svg image

When you have a p-value that's very low (conventionally < 0.05), that indicates the event was unlikely to have happened by random chance. Therefore we are inclined to think that Muriel has this special ability to detect when tea was poured before the milk, because if she was just randomly guessing she only had a 1.42% chance of succeeding. 

This example does not capture every nuance of a p-value, but it does provide the essence of it. Let's see how this concept applies with a sample mean and the normal distribution. 

## Two-Tailed Testing

Let's bring in the lightbulb dataset and calculate its mean, standard deviation, and count. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np 

X = pd.read_csv("https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/distribution/lightbulb_data.csv") \
    .squeeze()
X

mean, std, n = X.mean(), X.std(), X.count()
print("MEAN: ", mean)
print("STD: ", std)
print("n: ", n)

Based on this data, we can infer that there is a 95% probability the lightbulb will last approximately between 571.3 hours and 773.1 hours as shown below. 

In [ ]:
from matplotlib.patches import Polygon

fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

# .95 area 
a,b = norm.ppf(.025, mean, std), norm.ppf(.975, mean, std)
ix = np.linspace(a, b)
iy =  norm.pdf(ix, mean, std)
verts = [(a, 0), *zip(ix, iy), (b, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

# add text labels
plt.text(mean, .003, '.95', fontsize = 22, ha='center')
plt.text(a, norm.pdf(a,mean,std), round(a,2), fontsize = 16, ha='right', color='blue')
plt.text(b, norm.pdf(b,mean,std), round(b,2), fontsize = 16, ha='left', color='blue')


plt.show()

We can calculate that center range capturing the .95 area by using the inverse cumulative density function (the `ppf()` function in SciPy) as shown below. 

In [ ]:
from scipy.stats import norm

print(norm.ppf(.025, mean, std), norm.ppf(.975, mean, std))

Let's say an engineer made a design change and tested 31 light bulbs with this new design tweak. He happily reports that the mean in this new sample is 790. Now consider this... is this a coincidence? Let's look at this new mean compared to our current distribution of lightbulb lifespans. 

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

# .95 area 
a,b = norm.ppf(.025, mean, std), norm.ppf(.975, mean, std)
ix = np.linspace(a, b)
iy =  norm.pdf(ix, mean, std)
verts = [(a, 0), *zip(ix, iy), (b, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

# add text labels
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(mean, .003, '.95', fontsize = 22, ha='center')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

Hmm... take notice here that if we want to be 95% confident that the engineering change had an effect, logically the mean of the new sample must fall outside that .95 range (due to the central limit theorem). Therefore, we are inclined to believe with 95% confidence that the engineering change did improve the design.

Let's formalize these ideas a bit more.  We claim the **null hypothesis ($ H_0 $)** is the status quo: the mean is 672.2 and the engineering change had no effect. But the **alternative hypothesis ($ H_1 $)** is the mean *is not* 672.2 with the new engineering design, and we believe it did have an effect enough to reject the null hypothesis. 

$$
H_0: \mu = 672.2 
$$

$$
H_1: \mu \ne 672.2 
$$

Because this new mean of 790 does fall outside that 95% confidence range, we call it **statistically significant**, meaning it is unlikely to be coincidental enouugh that we can reject the null hypothesis and promote the alternative hypothesis. 

But *how unlikely* is it we would have observed 790 with the current lightbulb design? We can roughly say it falls outside the 95% "coincidence" range as shown above, but by how much? This is where the **p-value** once again comes in. Let's take a look at area left over in each tail, which is going to be $ .025 $. 

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

def plot_tail(a, b): 
    # p-value areas
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(.025, mean, std))
plot_tail(norm.ppf(.975, mean, std), mean+std*3)

# add text labels
plt.vlines(x = new_mean, ymin = norm.pdf(new_mean, mean, std), ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(555, .0001, '.025', fontsize = 12, ha='center')
plt.text(775, .0001, '.025', fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

However, this is not our p-value. The p-value has to capture any probability that is of equal or less value on both sides where we observed the new mean.  After all, we are trying to prove significance, and that includes anything that is equally or less likely to happen. Let's visualize just that area on both sides, which in total will be the p-value. 

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

def plot_tail(a, b): 
    # p-value areas
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(1.0 - norm.cdf(new_mean, mean, std), mean, std))
plot_tail(new_mean, mean+std*3)

# add text labels
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
print(one_side_p_value*2)
plt.text(530, .0005, round(one_side_p_value, 3), fontsize = 12, ha='center')
plt.text(800, .0005, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

So we are left  approximately with a p-value of $ 0.022 $, which is divided to both tails as $ 0.011 $. While trying to identify the ranges of these tails requires some deductive math and a bit of code, calculating the p-value itself can be done in two lines of code. We can simply calculate the area of the right tail using the CDF at the new sample mean (and subtract from 1.0 to get the right area, not the left), and then doubling it to take advantage of the symmetry of the left tail. 

In [ ]:
# area of right tail 
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# p-value of both tails (symmetrical)
p_value = p_value_right_tail*2 

print(p_value) # 0.02212358425605565

So practically speaking, how do we interpet this p-value? We say that the re-enginerred lightbulb life span of 790  is 2.2% likely to have happened by chance (including probability of equally or less likely events). If our threshold is 95% confidence, or a 5% level of significance, $ 0.022 $ is less than $ 0.05 $ so we can reject our null hypothesis, and promote the alternative hypothesis that our re-engineered lightbulb is an improvement. 

Alternatively, let's say the new lightbulb only improved the sample mean to 750. Notice below this would not be in the "statistically significant" range even though it is higher than the current mean  of 672.2. 

In [ ]:
from matplotlib.patches import Polygon

new_mean = 750

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

def plot_tail(a, b): 
    # p-value areas
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(.025, mean, std))
plot_tail(norm.ppf(.975, mean, std), mean+std*3)

# add text labels
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

plt.text(555, .0001, '.025', fontsize = 12, ha='center')
plt.text(775, .0001, '.025', fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

At a 95% threshold, we cannot attribute this lightbulb to perform significantly better than the current one because there's a good chance it was just by random luck and performs the same as the current lightbulb. How much of a chance? Well again, that is the p-value and we can calculate that. We find it is $ 0.13 $, and that is much higher than $ 0.05 $. Therefore we cannot reject the null hypothesis. 

In [ ]:
new_mean = 750 

# area of right tail 
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# p-value of both tails (symmetrical)
p_value = p_value_right_tail*2 

print(p_value) # 0.13072525593787487

And just for good measure, here is the p-value area visualized for a new sample mean of 750. 

In [ ]:
from matplotlib.patches import Polygon

new_mean = 750

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

def plot_tail(a, b): 
    # p-value areas
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(mean-std*3, norm.ppf(1.0 - norm.cdf(new_mean, mean, std), mean, std))
plot_tail(new_mean, mean+std*3)

# add text labels
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
print(one_side_p_value*2)
plt.text(555, .0015, round(one_side_p_value, 3), fontsize = 12, ha='center')
plt.text(775, .0015, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

## One-Tailed Testing

While not often as robust two-tailed testing, there is another way to perform hypothesis testing via **one-tailed testing**. We express null and alternative hypotheses as inequalities. we would be 

$$
H_0: \mu <= 672.2 
$$

$$
H_1: \mu > 672.2 
$$

Let's visualize the p-value for one-tailed testing as shown below.

In [ ]:
from matplotlib.patches import Polygon

new_mean = 790

# plot chart
fig, ax = plt.subplots()
x_range = np.arange(mean-std*3, mean+std*3, .01) 
ax.plot(x_range, norm.pdf(x_range, mean, std)) # bell curve 

def plot_tail(a, b): 
    # p-value areas
    ix = np.linspace(a, b)
    iy = norm.pdf(ix, mean, std)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)

plot_tail(new_mean, mean+std*3)

# add text labels
plt.vlines(x = new_mean, ymin = 0, ymax = .005,
           colors = 'red',
           label = r"\bar{x}_{new}")

one_side_p_value  = 1.0 - norm.cdf(new_mean, mean, std)
plt.text(800, .0005, round(one_side_p_value, 3), fontsize = 12, ha='left')
plt.text(new_mean, .005, round(new_mean,2), fontsize = 16, ha='center', va='bottom', color='red')


plt.show()

This simplifies the calculation of the p-value, by only including the tail that matches the alternative hypothesis. 

In [ ]:
new_mean = 790 

# area of right tail 
p_value = 1.0 -  norm.cdf(new_mean, mean, std)

print(p_value) # 0.011061792128027825

How does this affect whether we reject the null hypothesis? Ask yourself this: which one sets a higher threshold? You will notice that even when our objective is to show we may have decreased/increased something (the lifespan of the bulb), reframing our hypothesis to show any impact (greater or lesser) creates a higher significance threshold. If our significance threshold is a p-value of .05 or less, our one-tailed test had a much smaller p-value .011 as opposed to the two-tailed test, which was about double that at p-value .022.

For this reason, it is preferable to use the two-tailed test over one-tailed in many situations. It does not bias the test in one direction and it also sets a higher threshold for acceptance of the alternative hypothesis. 

## Dealing with Smaller Samples

As discussed in the previous section, if you have a smaller sample (n<31) then swap out the normal distribution with the T-distribution. For example, let's say the new lightbulb design had only a sample size of 5. We can calculate the two-tailed p-value as shown below. 

In [ ]:
from scipy.stats import t

new_mean = 790 
n = 5

# area of right tail 
p_value_right_tail = 1.0 -  t.cdf(new_mean, df=n-1, loc=mean, scale=std)

# p-value of both tails (symmetrical)
p_value = p_value_right_tail*2 

print(p_value) # 0.08401989317568481

Our p-value is much larger: 0.084. Because this does not pass our threshold of $ .05 $ we cannot reject the null hypothesis. 

## Exercise

An online gaming platform is trying to increase the playtime of its users with a new reward system. Typically, the average gamer will play for 95 minutes with a standard deviation of 20 minutes. After testing the new reward system with 100 randomly sampled gamers, they found the average play time increased to 125 miutes. Complete the code below using the two-tailed test to determine the p-value, and determine whether this is statistically significant with 95% confidence. 

In [ ]:
from scipy.stats import norm 

mean = 95
std = 20
new_mean = 125

# area of right tail 
p_value_right_tail = 1.0 -  ?

# p-value of both tails (symmetrical)
p_value = ?

print(p_value) 

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

The p-value is 0.1336, and since this is greater than 0.05 this fails to reject the null hypothesis. This means we cannot say the new reward system made any impact to the gamers' playtimes. 

In [ ]:
from scipy.stats import norm 

mean = 95
std = 20
new_mean = 125

# area of right tail 
p_value_right_tail = 1.0 -  norm.cdf(new_mean, mean, std)

# p-value of both tails (symmetrical)
p_value = p_value_right_tail*2 

print(p_value) # 0.13361440253771617